In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_json("/content/problems_data.jsonl", lines=True)

print(df.shape)
df.sample(5)

(4112, 8)


,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
1150,The Power of Substitution,A substitution cipher uses a substitution tabl...,The input starts with an integer $1 \le N \le ...,For each test case output a single integer $k$...,[{'input': '2 3 1 2 3 4 5 6 2 3 4 5 6 7 8 9 10...,hard,6.7,https://open.kattis.com/problems/substitution
3911,Counting Clauses,"It’s time for the annual 3-SAT competition, wh...",The input is a single instance of the 3-SAT pr...,Print “satisfactory” on a single\n line if ...,[{'input': '5 3 -1 2 3 -1 -2 3 1 -2 3 1 -2 -3 ...,easy,1.6,https://open.kattis.com/problems/countingclauses
2861,Purple Rain,"On close observation however, Prof. Nelson Rog...",The input consists of a single line containing...,"Print, on a single line, two space-separated i...","[{'input': 'BBRRBRRBRB', 'output': '3 7'}, {'i...",medium,3.7,https://open.kattis.com/problems/purplerain
1200,Flower Pots,Freddy is planning a garden party for this wee...,The input contains several test cases. Each te...,"For each test case, output a single line of te...","[{'input': 'II PP WF II VZ UP', 'output': 'YES...",hard,6.6,https://open.kattis.com/problems/flowerpots
298,Winter Roads,"Winter is coming, and the citizens of Winterfe...",There will be a single test case in the input....,"For each S a b w query, in order,\n output ...",[{'input': '3 4 1 2 3 2 3 3 2 1 1 1 2 1 6 S 1 ...,hard,8.3,https://open.kattis.com/problems/winterroads


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4112 non-null   object 
 2   input_description   4112 non-null   object 
 3   output_description  4112 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
dtypes: float64(1), object(7)
memory usage: 257.1+ KB


In [ ]:
df.isna().sum()

,0
title,0
description,0
input_description,0
output_description,0
sample_io,0
problem_class,0
problem_score,0
url,0


In [ ]:
df['problem_class'].value_counts(normalize=True)

,proportion
problem_class,
hard,0.472033
medium,0.341683
easy,0.186284


In [ ]:
df['problem_score'].describe()

,problem_score
count,4112.000000
mean,5.114689
std,2.177770
min,1.100000
25%,3.300000
50%,5.200000
75%,6.900000
max,9.700000


In [ ]:
px.histogram(df['problem_score'])

In [ ]:
text_cols = ['title', 'description', 'input_description', 'output_description']
df['full_text'] = df[text_cols].agg(' '.join, axis=1)
df['char_len'] = df['full_text'].str.len()
df['word_len'] = df['full_text'].str.split().str.len()

In [ ]:
df[['char_len',	'word_len','problem_score']].corr()

,char_len,word_len,problem_score
char_len,1.000000,0.993434,0.230039
word_len,0.993434,1.000000,0.233438
problem_score,0.230039,0.233438,1.000000


In [ ]:
df.groupby('problem_class')[['char_len', 'word_len']].mean()

,char_len,word_len
problem_class,,
easy,1289.954308,217.143603
hard,1761.594024,296.022154
medium,1619.276157,271.855516


In [ ]:
keywords = ['dp', 'graph', 'tree', 'recursion', 'segment', 'bit', 'modulo']

for kw in keywords:
    df[kw] = df['full_text'].str.contains(kw, case=False, regex=False)

In [ ]:
df.groupby('problem_class')[keywords].mean()

,dp,graph,tree,recursion,segment,bit,modulo
problem_class,,,,,,,
easy,0.016971,0.033943,0.030026,0.000000,0.019582,0.086162,0.009138
hard,0.029366,0.094796,0.078310,0.000515,0.046883,0.124163,0.063369
medium,0.010676,0.064057,0.061210,0.001423,0.022776,0.098221,0.033452


# try_1

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 11.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import xgboost as xgb
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
#from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_json("/content/problems_data.jsonl", lines=True)

def build_full_text(df):
    text_cols = ['title', 'description', 'input_description', 'output_description']
    return (
        df[text_cols]
        .fillna('')
        .agg(' '.join, axis=1)
        .str.lower()
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )

df['full_text'] = build_full_text(df)

le = LabelEncoder()
df['problem_class_enc'] = le.fit_transform(df['problem_class'])
print(le.classes_)

y_cls = df['problem_class_enc']
y_reg = df['problem_score']

['easy' 'hard' 'medium']


In [ ]:
def extract_stat_features(df):
    text = df['full_text']

    features = pd.DataFrame({
        'char_len': text.str.len(),
        'word_len': text.str.split().str.len(),
        'sentence_count': text.str.count(r'[.!?]') + 1,

        'digit_count': text.str.count(r'\d'),
        'math_symbol_count': text.str.count(r'[\+\-\*/=<>\^]'),

        # NEW (important)
        'log_char_len': np.log1p(text.str.len()),
        'log_word_len': np.log1p(text.str.split().str.len()),

        'constraint_count': text.str.count(r'(<=|>=|<|>)'),
        'power_count': text.str.count(r'\^'),
        'big_o_count': text.str.count(r'o\('),

        'newline_count': text.str.count(r'\n'),
        'colon_count': text.str.count(':'),

        'avg_word_len': text.str.len() / (text.str.split().str.len() + 1),
    })

    return features

KEYWORDS = {
    # Core paradigms
    'dp': ['dp', 'dynamic programming'],
    'graph': ['graph', 'tree', 'dag'],
    'greedy': ['greedy'],
    'binary_search': ['binary search'],
    'two_pointers': ['two pointers'],

    # Data structures
    'segment_tree': ['segment tree'],
    'fenwick': ['fenwick', 'bit'],
    'heap': ['heap', 'priority queue'],
    'disjoint_set': ['dsu', 'union find'],

    # Math / advanced
    'modulo': ['mod', 'modulo'],
    'combinatorics': ['combination', 'permutation'],
    'probability': ['probability', 'expected'],

    # Geometry / strings
    'geometry': ['geometry', 'convex'],
    'string_algo': ['kmp', 'z', 'suffix'],

    # Brute force indicators
    'bruteforce': ['brute', 'naive']
}

def extract_keyword_features(df):
    text = df['full_text']
    feats = {}

    for k, patterns in KEYWORDS.items():
        feats[f'{k}_present'] = text.apply(
            lambda x: any(p in x for p in patterns)
        ).astype(int)

        feats[f'{k}_count'] = text.apply(
            lambda x: sum(p in x for p in patterns)
        )

    return pd.DataFrame(feats)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=5,
    max_features=30000
)

#X_tfidf = tfidf.fit_transform(df['full_text'])

In [ ]:
X_stat = extract_stat_features(df)
X_kw = extract_keyword_features(df)
X_cls = pd.concat([X_stat,X_kw], axis=1)

In [ ]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix


In [ ]:
def logreg_objective(trial):
    penalty = 'l2'
    C = trial.suggest_float("C", 1e-4, 10.0, log=True)
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    class_weight = trial.suggest_categorical( "class_weight", [None, "balanced"])

    model = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(C=C, penalty='l2', solver='lbfgs', max_iter=3000, class_weight=class_weight, fit_intercept=fit_intercept, n_jobs=-1
    )) ])

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1s = []

    for tr, va in skf.split(X_cls, y_cls):
      model.fit(X_cls.iloc[tr], y_cls.iloc[tr])
      preds = model.predict(X_cls.iloc[va])
      f1s.append(f1_score(y_cls.iloc[va], preds, average='macro'))

    return np.mean(f1s)

study_cls = optuna.create_study(direction="maximize")
study_cls.optimize(logreg_objective, n_trials=100)

print("Best accuracy:", study_cls.best_value)
print("Best params:", study_cls.best_params)

[I 2026-01-06 12:47:35,256] A new study created in memory with name: no-name-f9634b50-83d7-462a-a013-1e0132cf4c8f
[I 2026-01-06 12:47:41,645] Trial 0 finished with value: 0.3816631689766804 and parameters: {'C': 0.00017780280605632806, 'fit_intercept': True, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.3816631689766804.
[I 2026-01-06 12:47:46,858] Trial 1 finished with value: 0.3688024961078077 and parameters: {'C': 0.00043093232997474965, 'fit_intercept': False, 'class_weight': None}. Best is trial 0 with value: 0.3816631689766804.
[I 2026-01-06 12:47:52,367] Trial 2 finished with value: 0.3590124661195668 and parameters: {'C': 0.17300874086421023, 'fit_intercept': True, 'class_weight': None}. Best is trial 0 with value: 0.3816631689766804.
[I 2026-01-06 12:47:58,024] Trial 3 finished with value: 0.37392988671597277 and parameters: {'C': 7.233170567481904, 'fit_intercept': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.3816631689766804.
[I 2026-01-06 1

Best accuracy: 0.40635186637397436
Best params: {'C': 0.02248467542960465, 'fit_intercept': True, 'class_weight': 'balanced'}


In [ ]:
 best_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(
        **study_cls.best_params,
        penalty='l2',
        solver='lbfgs',
        max_iter=3000,
        n_jobs=-1
    )) ])

X_tr, X_te, y_tr, y_te = train_test_split( X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls)

best_logreg.fit(X_tr, y_tr)
y_pred = best_logreg.predict(X_te)

print("Classification Report:")
print(classification_report(y_te, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_te, y_pred))
print(study_cls.best_params)

Classification Report:
              precision    recall  f1-score   support

        easy       0.29      0.68      0.41       153
        hard       0.59      0.45      0.51       389
      medium       0.38      0.23      0.28       281

    accuracy                           0.42       823
   macro avg       0.42      0.45      0.40       823
weighted avg       0.46      0.42      0.42       823

Confusion Matrix:
[[104  27  22]
 [129 176  84]
 [124  93  64]]
{'C': 0.02248467542960465, 'fit_intercept': True, 'class_weight': 'balanced'}


# xgboost

In [ ]:
def xgb_objective(trial):
    params = {
        'objective': 'multi:softmax',
        'num_class': len(np.unique(y_cls)),
        'eval_metric': 'mlogloss',
        'tree_method': 'hist',
        'verbosity': 0,

        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),

        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),

        'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 10.0, log=True),

        'n_estimators': 400,
        'random_state': 42
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1s = []

    for tr, va in skf.split(X_cls, y_cls):
        model = xgb.XGBClassifier(**params)
        model.fit(X_cls.iloc[tr], y_cls.iloc[tr])
        preds = model.predict(X_cls.iloc[va])
        f1s.append(f1_score(y_cls.iloc[va], preds, average='macro'))

    return np.mean(f1s)

study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(xgb_objective, n_trials=30)

print("Best macro-F1:", study_xgb.best_value)
print("Best params:", study_xgb.best_params)

[I 2026-01-06 12:32:10,623] A new study created in memory with name: no-name-17bc14ec-638d-4a69-9058-47542e51eebe
[I 2026-01-06 12:32:28,992] Trial 0 finished with value: 0.39119052512729 and parameters: {'learning_rate': 0.060060597444750057, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.7950158605654166, 'colsample_bytree': 0.7046238868555804, 'reg_alpha': 5.316471232270315e-05, 'reg_lambda': 0.002240037179408987}. Best is trial 0 with value: 0.39119052512729.
[I 2026-01-06 12:32:47,130] Trial 1 finished with value: 0.3988411173881142 and parameters: {'learning_rate': 0.06204476516878886, 'max_depth': 10, 'min_child_weight': 7, 'subsample': 0.7513083695091005, 'colsample_bytree': 0.6976255747337351, 'reg_alpha': 1.563434766831343, 'reg_lambda': 2.4006532205385235e-06}. Best is trial 1 with value: 0.3988411173881142.
[I 2026-01-06 12:32:54,499] Trial 2 finished with value: 0.40442037725337254 and parameters: {'learning_rate': 0.093617922192154, 'max_depth': 6, 'min_child_weigh

Best macro-F1: 0.4100112245649307
Best params: {'learning_rate': 0.017546057028404356, 'max_depth': 9, 'min_child_weight': 10, 'subsample': 0.7983661825920226, 'colsample_bytree': 0.862158917341956, 'reg_alpha': 0.06568950360695451, 'reg_lambda': 1.2089904955059225e-06}


In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(
    X_cls, y_cls,
    test_size=0.2,
    random_state=2,
    stratify=y_cls
)

final_xgb = xgb.XGBClassifier(
    **study_xgb.best_params,
    objective='multi:softmax',
    num_class=len(np.unique(y_cls)),
    eval_metric='mlogloss',
    tree_method='hist',
    random_state=42
)

final_xgb.fit(X_tr, y_tr)
y_pred = final_xgb.predict(X_te)

print("Classification Report:")
print(classification_report(y_te, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_te, y_pred))

Classification Report:
              precision    recall  f1-score   support

        easy       0.54      0.22      0.31       153
        hard       0.52      0.85      0.65       389
      medium       0.33      0.15      0.20       281

    accuracy                           0.49       823
   macro avg       0.46      0.41      0.39       823
weighted avg       0.46      0.49      0.43       823

Confusion Matrix:
[[ 33  82  38]
 [ 10 332  47]
 [ 18 222  41]]


# TF ITF logistic rex

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=5,
    max_features=30000
)

X_tfidf = tfidf.fit_transform(df['full_text'])

In [ ]:
from scipy.sparse import hstack, csr_matrix

X_cls = csr_matrix(hstack([
    X_tfidf,
    X_stat.values,
    X_kw.values
]))

y_cls = df['problem_class_enc'].values

In [ ]:
def logreg_objective(trial):
    C = trial.suggest_float("C", 1e-4, 10.0, log=True)
    class_weight = trial.suggest_categorical("class_weight", [None, "balanced"])

    model = Pipeline([
        ('scaler', StandardScaler(with_mean=False)),  # sparse-safe
        ('clf', LogisticRegression(
            C=C,
            solver='lbfgs',
            penalty='l2',
            max_iter=3000,
            class_weight=class_weight,
            n_jobs=-1
        ))
    ])

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1s = []

    for tr, va in skf.split(X_cls, y_cls):
        model.fit(X_cls[tr], y_cls[tr])
        preds = model.predict(X_cls[va])
        f1s.append(f1_score(y_cls[va], preds, average='macro'))

    return np.mean(f1s)

study = optuna.create_study(direction="maximize")
study.optimize(logreg_objective, n_trials=40)

print("Best macro-F1:", study.best_value)
print("Best params:", study.best_params)

[I 2026-01-06 13:03:48,998] A new study created in memory with name: no-name-436642ca-d2ba-4b24-b677-9ba67f97b094
[I 2026-01-06 13:04:01,039] Trial 0 finished with value: 0.4440200569808119 and parameters: {'C': 4.963384393533169, 'class_weight': None}. Best is trial 0 with value: 0.4440200569808119.
[I 2026-01-06 13:04:16,002] Trial 1 finished with value: 0.44569380757794363 and parameters: {'C': 0.0003574918215021117, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.44569380757794363.
[I 2026-01-06 13:04:32,015] Trial 2 finished with value: 0.4460451765150367 and parameters: {'C': 0.0009229036552481581, 'class_weight': 'balanced'}. Best is trial 2 with value: 0.4460451765150367.
[I 2026-01-06 13:04:49,708] Trial 3 finished with value: 0.4447694821697182 and parameters: {'C': 0.001998721341719635, 'class_weight': 'balanced'}. Best is trial 2 with value: 0.4460451765150367.
[I 2026-01-06 13:05:05,056] Trial 4 finished with value: 0.422078245412842 and parameters: {'C': 0.0012

Best macro-F1: 0.45412196925973997
Best params: {'C': 0.0001035369844298669, 'class_weight': 'balanced'}


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_tr, X_te, y_tr, y_te = train_test_split(
    X_cls, y_cls,
    test_size=0.2,
    random_state=42,
    stratify=y_cls
)

final_model = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(
        **study.best_params,
        solver='lbfgs',
        penalty='l2',
        max_iter=3000,
        n_jobs=-1
    ))
])

final_model.fit(X_tr, y_tr)
y_pred = final_model.predict(X_te)

print(classification_report(y_te, y_pred, target_names=le.classes_))
print(confusion_matrix(y_te, y_pred))

              precision    recall  f1-score   support

        easy       0.52      0.37      0.44       153
        hard       0.56      0.64      0.60       389
      medium       0.37      0.35      0.36       281

    accuracy                           0.49       823
   macro avg       0.48      0.45      0.46       823
weighted avg       0.49      0.49      0.49       823

[[ 57  38  58]
 [ 27 248 114]
 [ 25 157  99]]


In [ ]:
print(study.best_params)

{'C': 0.0001035369844298669, 'class_weight': 'balanced'}


# final training

## classification

In [ ]:
best_params = {'C': 0.0001035369844298669, 'class_weight': 'balanced'}